In [1]:
## imports
import pandas as pd
import numpy as np
import mne
import re
import os

from mne.preprocessing import (
    ICA, 
    create_ecg_epochs, 
    corrmap, 
    create_eog_epochs
)

# mne graphics config
# from PySide6 import QtCore, QtGui, QtWidgets, __version__ 

rawa = None
rawb = None

import datetime
print('IMPORTED MODULES today:', datetime.datetime.now())

# Removed cached function import
import sys
try:
    del sys.modules['rename_events']
except KeyError:
    pass

from rename_events import relabel_events


# setup

# SESSION AND POSNER CODES
session_id='m_07_01'
posner_id ='pos1a'
posner_id_b ='pos1b'

IMPORTED MODULES today: 2023-07-17 21:28:31.867673


In [3]:
%matplotlib qt

# Posner A

In [2]:
# Relabel stim events and load raw data
raw_a = relabel_events(session_id, posner_id)

Data file:  m_07_01/m_07_01_pos1a.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Found '1a' in file: m_07_01/07_1a_posner_task_2023-05-18_14h53.49.750/07_1a_posner_task_2023-05-18_14h53.49.750.csv
Found '1b' in file: m_07_01/07_1b_posner_task_2023-05-18_15h57.47.621/07_1b_posner_task_2023-05-18_15h57.47.621.csv
********** 1a
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Cropping annotations 2023-05-18 14:55:41.311271+00:00 - 2023-05-18 15:09:02.331271+00:00
  [0] Keeping  (2023-05-18 14:55:41.311271+00:00 - 2023-05-18 14:55:41.311271+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-05-18 14:56:56.723271+00:00 - 2023-05-18 14:56:56.723271+00:00 -> 75.412 - 75.412)
  [2] Keeping  (2023-05-18 

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [328] Keeping  (2023-05-18 15:06:27.752271+00:00 - 2023-05-18 15:06:27.752271+00:00 -> 646.441 - 646.441)
  [329] Keeping  (2023-05-18 15:06:31.322271+00:00 - 2023-05-18 15:06:31.322271+00:00 -> 650.011 - 650.011)
  [330] Keeping  (2023-05-18 15:06:31.422271+00:00 - 2023-05-18 15:06:31.422271+00:00 -> 650.111 - 650.111)
  [331] Keeping  (2023-05-18 15:06:32.762271+00:00 - 2023-05-18 15:06:32.762271+00:00 -> 651.451 - 651.451)
  [332] Keeping  (2023-05-18 15:06:34.262271+00:00 - 2023-05-18 15:06:34.262271+00:00 -> 652.951 - 652.951)
  [333] Keeping  (2023-05-18 15:06:37.232271+00:00 - 2023-05-18 15:06:37.232271+00:00 -> 655.921 - 655.921)
  [334] Keeping  (2023-05-18 15:06:37.332271+00:00 - 2023-05-18 15:06:37.332271+00:00 -> 656.021 - 656.021)
  [335] Keeping  (2023-05-18 15:06:39.262271+00:00 - 2023-05-18 15:06:39.262271+00:00 -> 657.951 - 657.951)
  [336] Keeping  (2023-05-18 15:06:40.762271+00:00 - 2023-05-18 15:06:40.762271+00:00 -> 659.451 - 659.451)
  [337] Keeping  (2023-05-18

In [4]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events, event_id = mne.events_from_annotations(raw_a, event_map)

e_a = events
# all rows with fixation (code 40)
e_40_a = e_a[e_a[:, 2] == 40]
# all rows with trial end (code 101)
e_101_a = e_a[e_a[:,2]==101]

a_tmin= e_40_a[0][0]/1000
a_tmax= e_101_a[-1][0]/1000

# Crop and make a copy and filter
filt_raw_a = raw_a.crop(tmin=a_tmin, tmax=a_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_a = mne.Epochs(filt_raw_a, 
                    e_a, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
413 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 413 events and 701 original time points ...
4 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.7s finished


In [50]:
raw_a.plot()

Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [5]:
# Fitting ICA
ica = ICA(n_components=len(filt_raw_a.ch_names)-1, max_iter="auto", random_state=999)
ica.fit(filt_raw_a)
ica

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Fitting ICA took 61.1s.


Method,fastica
Fit,88 iterations on raw data (686561 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [9]:
ica.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [ ]:
raw_a.load_data()
ica.plot_sources(raw_a)

Creating RawArray with float64 data, n_channels=63, n_times=686561
    Range : 101918 ... 788478 =    101.918 ...   788.478 secs
Ready.
Using qt as 2D backend.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
343 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
343 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
343 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
343 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
343 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
343 matching events found
No baseline correction applied
0 projection items ac

In [10]:
print(ica.exclude)

[41, 56, 50, 45, 28, 31, 0, 1, 2, 6, 5, 7, 8, 12, 13, 11, 10]


In [11]:
reconstr_raw = raw_a.copy()
# Apply ica to copy
ica.apply(reconstr_raw)
# Plot
raw_a.plot(title="Raw data")
reconstr_raw.plot(title="ICA applied")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 17 ICA components
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [12]:
# Save file
file_name = session_id+'_'+posner_id+'.vhdr'
output_dir='cleaned'

print(file_name)
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir, session_id)
print(session_dir)

if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name) 
mne.export.export_raw(session_dir+'/'+file_name, reconstr_raw, verbose=1)

m_07_01_pos1a.vhdr
/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_07_01

**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_07_01/m_07_01_pos1a.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [13]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_07_01
├── m_07_01_pos1a.eeg
├── m_07_01_pos1a.vhdr
└── m_07_01_pos1a.vmrk

1 directory, 3 files


In [14]:
del raw_a
del reconstr_raw
del ica
del filt_raw_a

# Posner B

In [15]:
# Relabel stim events and load raw data
raw_b = relabel_events(session_id, posner_id_b)

Data file:  m_07_01/m_07_01_pos1b.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S111', 'Stimulus/S117']
Found '1a' in file: m_07_01/07_1a_posner_task_2023-05-18_14h53.49.750/07_1a_posner_task_2023-05-18_14h53.49.750.csv
Found '1b' in file: m_07_01/07_1b_posner_task_2023-05-18_15h57.47.621/07_1b_posner_task_2023-05-18_15h57.47.621.csv
********** 1b
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S111', 'Stimulus/S117']
Cropping annotations 2023-05-18 16:00:53.123826+00:00 - 2023-05-18 16:13:35.583826+00:00
  [0] Keeping  (2023-05-18 16:00:53.123826+00:00 - 2023-05-18 16:00:53.123826+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-05-18 16

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [394] Keeping  (2023-05-18 16:12:51.608826+00:00 - 2023-05-18 16:12:51.608826+00:00 -> 718.485 - 718.485)
  [395] Keeping  (2023-05-18 16:12:53.108826+00:00 - 2023-05-18 16:12:53.108826+00:00 -> 719.985 - 719.985)
  [396] Keeping  (2023-05-18 16:12:55.878826+00:00 - 2023-05-18 16:12:55.878826+00:00 -> 722.755 - 722.755)
  [397] Keeping  (2023-05-18 16:12:55.978826+00:00 - 2023-05-18 16:12:55.978826+00:00 -> 722.855 - 722.855)
  [398] Keeping  (2023-05-18 16:12:58.118826+00:00 - 2023-05-18 16:12:58.118826+00:00 -> 724.995 - 724.995)
  [399] Keeping  (2023-05-18 16:12:59.618826+00:00 - 2023-05-18 16:12:59.618826+00:00 -> 726.495 - 726.495)
  [400] Keeping  (2023-05-18 16:13:02.368826+00:00 - 2023-05-18 16:13:02.368826+00:00 -> 729.245 - 729.245)
  [401] Keeping  (2023-05-18 16:13:02.468826+00:00 - 2023-05-18 16:13:02.468826+00:00 -> 729.345 - 729.345)
  [402] Keeping  (2023-05-18 16:13:04.618826+00:00 - 2023-05-18 16:13:04.618826+00:00 -> 731.495 - 731.495)
  [403] Keeping  (2023-05-18

In [16]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events_b, event_id_b = mne.events_from_annotations(raw_b, event_map)

e_b = events_b
# all rows with fixation (code 40)
e_40_b = e_b[e_b[:, 2] == 40]
# all rows with trial end (code 101)
e_101_b = e_b[e_b[:,2]==101]

b_tmin= e_40_b[0][0]/1000
b_tmax= e_101_b[-1][0]/1000

# Crop and make a copy and filter
filt_raw_b = raw_b.crop(tmin=b_tmin, tmax=b_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_b = mne.Epochs(filt_raw_b, 
                    e_b, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '117', '20', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
414 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 414 events and 701 original time points ...
6 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.6s finished


In [23]:
raw_b.plot()

Using pyopengl with version 3.1.6


Channels marked as bad:
none


In [17]:
# Fitting ICA
ica_b = ICA(n_components=len(filt_raw_b.ch_names)-1, max_iter="auto", random_state=999)
ica_b.fit(filt_raw_b, verbose='DEBUG')
ica_b

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Cropping annotations 2023-05-18 16:02:10.554826+00:00 - 2023-05-18 16:13:19.746826+00:00
  [0] Keeping  (2023-05-18 16:02:10.554826+00:00 - 2023-05-18 16:02:10.554826+00:00 -> 77.431 - 77.431)
  [1] Keeping  (2023-05-18 16:02:12.054826+00:00 - 2023-05-18 16:02:12.054826+00:00 -> 78.931 - 78.931)
  [2] Keeping  (2023-05-18 16:02:15.804826+00:00 - 2023-05-18 16:02:15.804826+00:00 -> 82.681 - 82.681)
  [3] Keeping  (2023-05-18 16:02:15.904826+00:00 - 2023-05-18 16:02:15.904826+00:00 -> 82.781 - 82.781)
  [4] Keeping  (2023-05-18 16:02:17.054826+00:00 - 2023-05-18 16:02:17.054826+00:00 -> 83.931 - 83.931)
  [5] Keeping  (2023-05-18 16:02:18.554826+00:00 - 2023-05-18 16:02:18.554826+00:00 -> 85.431 - 85.431)
  [6] Keeping  (2023-05-18 16:02:22.434826+00:00 - 2023-05-18 16:02:22.434826+00:00 -> 89.311 - 89.311)
  [7] Keeping  (2023-05-18 16:02:22.534826+00:00 - 2023-05-18 16:02

Method,fastica
Fit,77 iterations on raw data (669192 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [18]:
ica_b.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [ ]:
raw_b.load_data()
ica_b.plot_sources(raw_b)

Creating RawArray with float64 data, n_channels=63, n_times=669192
    Range : 77431 ... 746622 =     77.431 ...   746.622 secs
Ready.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
334 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
334 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
334 matching events found
No baseline correction applied
0 projection items activated


In [19]:
ica_b.exclude

[61, 43, 58, 51, 22, 23, 0, 1, 5, 6, 8, 25, 26, 27, 32, 31]

In [20]:
reconstr_raw_b = raw_b.copy()
# Apply ica to copy
ica_b.apply(reconstr_raw_b)
# Plot and compare
raw_b.plot(title="Raw data (posner b)")
reconstr_raw_b.plot(title="ICA applied (posner b)")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 16 ICA components
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [21]:
# Save file
file_name_b = session_id+'_'+posner_id_b+'.vhdr'
output_dir_b='cleaned'
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir_b, session_id)
if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name_b) 
mne.export.export_raw(session_dir+'/'+file_name_b, reconstr_raw_b, verbose=1)


**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_07_01/m_07_01_pos1b.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [22]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_07_01
├── m_07_01_pos1a.eeg
├── m_07_01_pos1a.vhdr
├── m_07_01_pos1a.vmrk
├── m_07_01_pos1b.eeg
├── m_07_01_pos1b.vhdr
└── m_07_01_pos1b.vmrk

1 directory, 6 files


# ...

In [23]:
del raw_b
del reconstr_raw_b
del ica_b
del filt_raw_b